In [1]:
'''Read a single xml file.
Generate lists of asym_ids, seq_ids, comp_ids, atom_ids.
Classify whether Chain is RNA or Protein chain and generate corresponding lists.
Take in a RNA Chain Asym_id and a protein chain asym_id.
Find interaction of each compound of RNA chain with each compound of protein chain.
This computation will be done at atomic level.
Compress this by taking mean of the above matrix.
Generate a matrix that shows the interaction of one Protein chain with one RNA chain
Do so for each pair of RNA chain asym_id and protein chain asym_id of the complex.'''

'Read a single xml file.\nGenerate lists of asym_ids, seq_ids, comp_ids, atom_ids.\nClassify whether Chain is RNA or Protein chain and generate corresponding lists.\nTake in a RNA Chain Asym_id and a protein chain asym_id.\nFind interaction of each compound of RNA chain with each compound of protein chain.\nThis computation will be done at atomic level.\nCompress this by taking mean of the above matrix.\nGenerate a matrix that shows the interaction of one Protein chain with one RNA chain\nDo so for each pair of RNA chain asym_id and protein chain asym_id of the complex.'

In [2]:
#importing necessary libraries

from lxml import etree #Using the lxml library
import numpy as np #For creating ndarrays
import scipy # For finding distances
import pickle # For saving interaction dict on disc
from scipy import spatial # For finding distances
from collections import namedtuple # For creating a custom tuple of asym_id, seq_id and comp_id as key

In [3]:
# function takes in the filename of the pdb file as a parameter and returns lists correspoding to x,y,z co-ordinates
# asymId, atomId, CompoundId and SequenceId along with number of atoms in the complex called indexes
# Using auth_versions

def readPdbxml(filename):
    pdbx = etree.parse(filename) #Directly read the xml file
    root = pdbx.getroot() #Get the root and use it as a reference for further operations.
    
    #finding at what index normal ATOMs end and poly sequences start coming
    
    indexes = len(root[0]) # Total valid ATOM ids 
    final_indexes = indexes
    
    x_coord = []
    y_coord = []
    z_coord = []
    asym_id = []
    atom_id = []
    comp_id = []
    seq_id = []
    
    to_ignore_comp_id = ['MG','NA','HOH']
    
    for each_i in range(indexes):
        
        if(root[0][each_i][6].text) in to_ignore_comp_id:
            final_indexes-=1
            continue
        
        x_coord.append(float(root[0][each_i][1].text))
        y_coord.append(float(root[0][each_i][2].text))
        z_coord.append(float(root[0][each_i][3].text))
        asym_id.append(root[0][each_i][4].text)
        atom_id.append(root[0][each_i][5].text)
        comp_id.append(root[0][each_i][6].text)
        seq_id.append(root[0][each_i][7].text)
        
    return x_coord,y_coord,z_coord,asym_id,atom_id,comp_id,seq_id,final_indexes
    
    

In [4]:
# function to create a dictionary that has (asym_id,seq_id,comp_id) as key and a list of lists as values
# where each list has 3 values,ie, x,y,z co-ordinate

def seq_dict(seq_id, x_coord, y_coord, z_coord,indexes):
    
    seq_data = {}
    
    for each_index in range(indexes):
      
        level1_key = namedtuple("level1_key",["asym_id", "seq_id", "comp_id"])
        key = level1_key(asym_id[each_index],int(seq_id[each_index]),comp_id[each_index])
        location = np.asarray([x_coord[each_index], y_coord[each_index], z_coord[each_index]])
        seq_data.setdefault(key, []).append(location)
    
    return seq_data
    

In [5]:
# Function takes in a dict with list of numpy arrays as values and the level1 key as key and returns 
# a dict with n*3 numpy arrays ( no more lists) as values and same keys

def lists_to_ndarrays(seq_data):

    dict_ndarr_seq = {}

    for key in seq_data:
        #store as nd array
        value = np.array(seq_data[key])
        dict_ndarr_seq[key] = value
    
    return dict_ndarr_seq

In [6]:
''' Parses through all the asym_ids and classify them as RNA or Protein
chains. Returns two lists. One having asym_ids of RNA chains other of Protein chains'''


def rna_or_protein(asym_id, comp_id):
    
    #ADE, URI, GUA,CYT
    rna_compound = ['A','U','G','C'] # If the asym_id is a rna, all its compounds must be one of these
    unique_asym_ids = set(asym_id) # Gives the unique asym_ids in the complex
    
    rna_chain = [] # Initialising empty list that will later hold all asym_ids that are RNA chains
    protein_chain = [] # Initialising empty list that will later hold all asym_ids that are protein chains

    for each_asym in unique_asym_ids:
    
        flag = 0; # the variable detects if even one compound is not one of rna and helps classify it as protein
    
        for asym,each_comp in zip(asym_id,comp_id):
        
            if(each_asym == asym):
            
                if(each_comp not in rna_compound):
                
                    flag = 1
                    break
        
        if(flag):
        
            protein_chain.append(each_asym)
        
        else:
        
            rna_chain.append(each_asym)
        

    return protein_chain, rna_chain
    

In [7]:
'''Function removes duplicate values for same key from asym_id dict. Helper function'''

def remove_dup(asym_id_dict):
    
    dict_comp_in_asym = {}
    
    for key in asym_id_dict:
        
        unique = set(asym_id_dict[key])
        unique = tuple(unique)
        dict_comp_in_asym.setdefault(key, []).append(unique)
        
    return dict_comp_in_asym

In [8]:
'''Function creates a dict that has asym_id as key and a tuple of (seq_id, comp_id) as value'''

def construct_asym_id_dict(asym_id, seq_id, comp_id):
    
    asym_id_dict = {}
    
    for each_index in range(indexes):
      
        key = asym_id[each_index]
        val = (seq_id[each_index], comp_id[each_index])
        asym_id_dict.setdefault(key, []).append(val)
        
        dict_comp_in_asym = remove_dup(asym_id_dict)
    
    return dict_comp_in_asym
    

In [9]:
'''Function takes in two sets of (asym_id, seq_id, comp_id) and finds pairwise distance between each atom.
   Also, takes in the dictionary that has atom co-ordinates'''

def pairwise_inter_compound(key1, key2,dict_ndarr_seq):
    
    compound_interaction = scipy.spatial.distance.cdist(dict_ndarr_seq[key1], dict_ndarr_seq[key2],'euclidean')
    
    return compound_interaction

In [10]:
'''Function takes in asym_id of a protein_chain(m compounds), asym_id of a rna_chain(n compounds) and 
also the pre-computed dictionary that has asym_id as key and seq_id and comp_id as value. It then returns
m*n matrices where each matrix is interaction between one compound of protein_chain with one compound of
rna_chain'''

def pairwise_inter_chain(rna_asym_id, protein_asym_id, dict_comp_in_asym,dict_ndarr_seq):
    
    #need to construct appropraite keys to send to pariwsie_inter_compound
    
    #create a dict with asym_id as key and seq_id and comp_id as value
    
    compounds_in_rna = dict_comp_in_asym[rna_asym_id]
    compounds_in_protein = dict_comp_in_asym[protein_asym_id]
    
    chain_interaction = {}
    
    for nesting in  compounds_in_rna:
        
        for each_rna_compound in nesting:
        
        
            key1 = (rna_asym_id, int(each_rna_compound[0]), each_rna_compound[1])
        
            for nesting_2 in compounds_in_protein:
                
                for each_protein_compound in nesting_2:
            
                    key2 = (protein_asym_id, int(each_protein_compound[0]), each_protein_compound[1])
            
                    compound_interaction = pairwise_inter_compound(key1,key2,dict_ndarr_seq)
            
                    level2_key = namedtuple("level2_key",["rna_comp", "protein_comp"])
                    key = level2_key(key1,key2)
            
                    chain_interaction.setdefault(key, []).append(compound_interaction)
            
    return chain_interaction
            
    

In [11]:
'''Function takes in the dense chain_interaction matrix. It finds the mean of the matrix of interaction between
a pair of compounds. Then returns a mean chain interaction dict that is actually a matrix of dimensions
Number of compounds in RNA chain * Number of compounds in Protein chain'''

def chain_interaction_compression(chain_interaction):
    
    mean_chain_interaction = {}

    for each_compound_pair in chain_interaction:
    
        mean_chain_interaction[each_compound_pair] = np.mean(chain_interaction[each_compound_pair][0])
        
    return mean_chain_interaction

In [12]:
# to generate mean_chain_interaction for each RNA chain and protein chain pair

def construct_complex_rna_protein_interaction(rna_chain, protein_chain, dict_comp_in_asym,dict_ndarr_seq):
    
    complex_rna_protein_interaction = {}
    
    for each_rna_chain in rna_chain:
        
        for each_protein_chain in protein_chain:
            
            chain_interaction = pairwise_inter_chain(each_rna_chain, each_protein_chain, dict_comp_in_asym,dict_ndarr_seq)
            mean_chain_interaction = chain_interaction_compression(chain_interaction)
            
            key = (each_rna_chain, each_protein_chain)
            value = mean_chain_interaction
            
            complex_rna_protein_interaction[key] = value
            
    return complex_rna_protein_interaction

In [14]:
cd dataset/

/home/arrayslayer/Documents/Acads/2-2/SOP/dataset


In [15]:
# Function calls

x_coord,y_coord,z_coord,asym_id,atom_id,comp_id,seq_id,indexes = readPdbxml('1wsu.xml')
seq_data = seq_dict(seq_id,x_coord,y_coord,z_coord,indexes)
dict_ndarr_seq = lists_to_ndarrays(seq_data)
dict_comp_in_asym = construct_asym_id_dict(asym_id, seq_id, comp_id)


protein_chain, rna_chain = rna_or_protein(asym_id,comp_id)

complex_rna_protein_interaction = construct_complex_rna_protein_interaction(rna_chain, protein_chain, dict_comp_in_asym,dict_ndarr_seq)

In [16]:
protein_chain

['B', 'C', 'A', 'D']

In [17]:
rna_chain

['E', 'G', 'F']

In [18]:
type(complex_rna_protein_interaction)

dict

In [19]:
for key in complex_rna_protein_interaction:
    print(key)

('E', 'B')
('E', 'C')
('E', 'A')
('E', 'D')
('G', 'B')
('G', 'C')
('G', 'A')
('G', 'D')
('F', 'B')
('F', 'C')
('F', 'A')
('F', 'D')


In [20]:
complex_rna_protein_interaction[('E','B')]

{level2_key(rna_comp=('E', 14, 'G'), protein_comp=('B', 556, 'TYR')): 60.572068454627576,
 level2_key(rna_comp=('E', 14, 'G'), protein_comp=('B', 519, 'LEU')): 54.414875992032641,
 level2_key(rna_comp=('E', 14, 'G'), protein_comp=('B', 630, 'VAL')): 58.687901744347428,
 level2_key(rna_comp=('E', 14, 'G'), protein_comp=('B', 622, 'THR')): 58.396625055667876,
 level2_key(rna_comp=('E', 14, 'G'), protein_comp=('B', 606, 'ARG')): 47.741177980293834,
 level2_key(rna_comp=('E', 14, 'G'), protein_comp=('B', 594, 'GLY')): 51.2328279213108,
 level2_key(rna_comp=('E', 14, 'G'), protein_comp=('B', 624, 'ARG')): 59.869720140856145,
 level2_key(rna_comp=('E', 14, 'G'), protein_comp=('B', 562, 'VAL')): 54.554102681332061,
 level2_key(rna_comp=('E', 14, 'G'), protein_comp=('B', 617, 'ASP')): 59.762608227472079,
 level2_key(rna_comp=('E', 14, 'G'), protein_comp=('B', 579, 'GLU')): 46.921489295377732,
 level2_key(rna_comp=('E', 14, 'G'), protein_comp=('B', 538, 'GLU')): 47.148288620275544,
 level2_key(